In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
    !pip install -q transformers
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/MyDrive/gan_bert_for_cpv')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.9 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
from transformers import logging
logging.set_verbosity_error()
import torch
from create_dataset import CreateDataset
from process_data import ProcessData
from utils import *
from torch.utils.data import DataLoader
from run_model import RunModel
from models import BertGenerator, BertDiscriminator
from transformers import AutoTokenizer, AutoModel, AutoConfig

In [3]:
from models import *

In [4]:
input_df = pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_dutch_ted1.csv")

In [5]:
input_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Korte beschrijving aanbesteding',
       'Naam Aanbestedende dienst', 'CPV-code', 'Omschrijving',
       'Omschrijving standaard', 'Publicatiedatum', 'afdeling', 'groep',
       'klasse', 'categorie'],
      dtype='object')

In [6]:
label_column = "afdeling"
text_column = "Korte beschrijving aanbesteding"

In [7]:
vc = input_df[label_column].value_counts()
down_sample_vc = vc[vc.values < 1500]
input_df = input_df[input_df[label_column].isin(down_sample_vc.index)]

In [ ]:
# down_sample_vc

In [8]:
running_device = torch.device("cpu")

In [9]:
prd = ProcessData(input_df, label_column, text_column, sample_num=1000, aug_limit=500,test_per_label_num=30, drop_limit=50,
                  balanced_test=True)
test_df, suf_df, insuf_df, label_encoder, onehot = prd.process()
train_df = pd.concat([suf_df, insuf_df]).sample(frac=1)
value_counts = insuf_df[label_column].value_counts()
model_runner = RunModel(running_device)

After removing duplicate sentence: 16897
After drop cpv with low observations we have : 16803
The num of class is  30
The number of total data is:  15992
Total number of test data is:  900
Total number of train data is:  15092


In [12]:
train_df.to_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/sampled_train.csv")

In [11]:
test_df.to_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/sampled_test.csv")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
config = AutoConfig.from_pretrained("GroNLP/bert-base-dutch-cased", output_hidden_states=True)
bert = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased", config=config).to(running_device)

In [ ]:
cd = CreateDataset(text_column, label_column, label_encoder, onehot, 1, running_device, max_length=120)
test_dataset = cd.create(test_df)
# suf_dataset = cd.create(suf_df)
test_dl = DataLoader(test_dataset, shuffle=True, batch_size=20)
insuf_ds_dict = cd.create_label_dataset(insuf_df)
insuf_ds_list = list(insuf_ds_dict.values())
original_insuf_dataset = torch.utils.data.ConcatDataset(insuf_ds_list)
# original_train_dataset = torch.utils.data.ConcatDataset([suf_dataset, original_insuf_dataset])
balanced_num = 500
batch_size = 20
gan_aug_datasets = []
copy_aug_datasets=[]
for label in insuf_ds_dict:
    sub_ds = insuf_ds_dict[label]
    # balanced_num=3*len(sub_ds)
    generator = BertGenerator(bert)
    discriminator = BertDiscriminator(bert)
    enrich_num = balanced_num - len(sub_ds)
    enrich_batch = int(enrich_num / batch_size)*2
    source_train = DataLoader(sub_ds, shuffle=True, batch_size=20)
    generator, discriminator = model_runner.generator_trainer(generator, discriminator,
                                  source_train, source_train,
                                  batch_num=20, n_epochs=20, 
                                  glr=3e-6, dlr=3e-6)
    one_data, one_label = sub_ds[0]
    size = (batch_size, one_data.size(0), one_data.size(1))
    gan_aug_dataset = model_runner.generate_aug_data(generator, discriminator, 
                              one_label, size=size,batch_num=enrich_batch)
    print(len(sub_ds))
    print(len(gan_aug_dataset))
    gan_aug_datasets.append(gan_aug_dataset)

Getting bert hidden states from layer: 1: 100%|██████████| 93/93 [00:01<00:00, 80.84it/s]


0 discriminator loss:  1.345852130651474 generator loss : 0.07976900367066264 real acc1:  1.0 fake middle confidence: 0.5077265302340189
1 discriminator loss:  1.3268953800201415 generator loss : 0.2358945593237877 real acc1:  1.0 fake middle confidence: 0.5148588418960571
2 discriminator loss:  1.3175355434417724 generator loss : 0.35975696742534635 real acc1:  1.0 fake middle confidence: 0.5271965066591898
3 discriminator loss:  1.3159965574741364 generator loss : 0.44236173778772353 real acc1:  1.0 fake middle confidence: 0.5394246379534403
4 discriminator loss:  1.3169323742389678 generator loss : 0.4931551769375801 real acc1:  1.0 fake middle confidence: 0.5513565142949423
5 discriminator loss:  1.3145229279994965 generator loss : 0.5231419742107392 real acc1:  1.0 fake middle confidence: 0.5602923929691315
6 discriminator loss:  1.3080525636672973 generator loss : 0.5379898130893708 real acc1:  1.0 fake middle confidence: 0.5655902922153473
7 discriminator loss:  1.30372626781463

In [ ]:
gan_aug_train_dataset = torch.utils.data.ConcatDataset(gan_aug_datasets + [original_insuf_dataset])
gan_train_dl = DataLoader(gan_aug_train_dataset, shuffle=True, batch_size=20)

In [ ]:
classifier = HiddenClassifier1(prd.num_class, bert)
classifier = model_runner.train_classifier(gan_train_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recal=evaluate(labels, predictions)

  0%|          | 12/4631 [00:04<31:09,  2.47it/s]

In [ ]:
def save_model(path, model):
    print("Saving model to %s" % path)
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(path)

In [ ]:
save_model(classifier,"/content/drive/MyDrive/gan_bert_for_cpv/output/department")

In [ ]:
classifier = model_runner.train_classifier(gan_train_dl, test_dl, classifier, lr=2e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recal=evaluate(labels, predictions)

100%|██████████| 3983/3983 [20:15<00:00,  3.28it/s]


0 train loss:  0.5218340074339521 val acc:  0.6125000109275182


100%|██████████| 3983/3983 [20:16<00:00,  3.27it/s]


1 train loss:  0.47174254646456976 val acc:  0.6291666800777117
Total ACC: 0.629
F1: 0.610
Precision: 0.615
Recall: 0.629


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import matplotlib.pyplot  as plt
plt.plot(acc_lists,label="ACC")
plt.plot(f1_lists,label="f1 score")
plt.plot(precision_lists,label="Precision")
plt.plot(recall_lists,label="Recall")
plt.title("Data Augmentation for different layers")
plt.legend()
plt.show()

In [ ]:
print("acc",acc_lists)
print("f1",f1_lists)
print("precision",precision_lists)
print("recall",recall_lists)

In [ ]:
from google. colab import runtime 
runtime. unassign()

# Gan enriched data

In [ ]:
copy_aug_insuf_dataset = []
for label in insuf_ds_dict:
    ds = insuf_ds_dict[label]
    aug_insuf_ds = [ds[i % len(ds)] for i in range(balanced_num)]
    copy_aug_insuf_dataset += aug_insuf_ds

NameError: ignored

In [ ]:
copy_aug_train_dataset = torch.utils.data.ConcatDataset(copy_aug_insuf_dataset + [original_train_dataset])

# Only copy enriched dataset

In [ ]:
copy_train_dl = DataLoader(copy_aug_train_dataset, shuffle=True, batch_size=20)
classifier = HiddenClassifier9(prd.num_class, bert).to(running_device)
classifier = model_runner.train_classifier(copy_train_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
evaluate(labels, predictions, prd.label_encoder, prd.num_class, value_counts)

# Hybrid enriched train dataset; copy+gan augmented

In [ ]:
hybrid_train_dataset = torch.utils.data.ConcatDataset(
    gan_aug_datasets + copy_aug_insuf_dataset + [original_train_dataset])
hybrid_train_dl = DataLoader(hybrid_train_dataset, shuffle=True, batch_size=20)
classifier = HiddenClassifier9(prd.num_class, bert).to(running_device)
classifier = model_runner.train_classifier(hybrid_train_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall = evaluate(labels, predictions, prd.label_encoder, prd.num_class, value_counts)

# Only original dataset training

In [ ]:
original_dl = DataLoader(original_train_dataset, shuffle=True, batch_size=20)
classifier = HiddenClassifier1(prd.num_class, bert).to(running_device)
classifier = model_runner.train_classifier(original_dl, test_dl, classifier, lr=1e-5, n_epochs=6)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall = evaluate(labels, predictions)

0 train loss:  2.9717725387587675 val acc:  0.3047619096580006
1 train loss:  1.9207754454576653 val acc:  0.39761905443100704
2 train loss:  1.440107233235131 val acc:  0.5261904895305634
3 train loss:  1.1035037373308876 val acc:  0.5464285817884263
4 train loss:  0.8879791532007772 val acc:  0.5714285834914162
5 train loss:  0.7403410421226357 val acc:  0.5845238211609068
Total ACC: 0.585
F1: 0.525
Precision: 0.491
Recall: 0.585


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
